<a href="https://colab.research.google.com/github/rafavidal1709/projeto-aplicado-iii/blob/main/04%20-%20Recomendando%20textos%20semelhantes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Vamos provar que nosso algorítimo de recomendação aplicado em triagem de denúncias funciona perfeitamente para recomendação de textos. Para tal utilizaremos o **BERTimbau**, o **cosine similarity** e o **output mean** como modelo, método de comparação e técnica de embedding, respectivamente, pois foram as técnicas que mostraram melhor acurácia.

**Passo 1**

Vamos iniciar com o *dataframe* "dataset jornal original.json", que se encontra disponível para [download no GitHub](https://github.com/rafavidal1709/projeto-aplicado-iii/blob/main/dataset%20jornal%20original.json). Basta executar o código abaixo e escolhê-lo para upload.

In [1]:
import json
from google.colab import files

# Carrega o dataset
def upload_dataset():
  uploaded = files.upload()
  return json.loads(uploaded[list(uploaded.keys())[0]].decode('utf-8'))

df = upload_dataset()

Saving dataset jornal original.json to dataset jornal original.json


**Passo 2**

Vamos copiar a classe **BertimbauEmbedding** já utilizada no "[01 - Modelos de embedding.ipynb](https://github.com/rafavidal1709/projeto-aplicado-iii/blob/main/01%20-%20Modelos%20de%20embedding.ipynb)", que é responsável por converter o texto em um vetor no processo chamado de embedding. Importante lembrar que o ambiente deve estar configurado para GPU.

In [2]:
import torch
from transformers import BertTokenizer, BertModel

class BertimbauEmbedding:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Verificar se a GPU está disponível
        torch.cuda.empty_cache()  # Liberar memória da GPU, se necessário
        self.tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-large-portuguese-cased")  # Carregar o modelo e o tokenizador BERTimbau-large
        self.model = BertModel.from_pretrained("neuralmind/bert-large-portuguese-cased").to(self.device)

    def process_text(self, text):
        with torch.no_grad():  # Desabilitar o cálculo de gradientes
            # Reduzir o comprimento máximo, se possível
            inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)

            # Enviar os inputs para a GPU
            inputs = {key: value.to(self.device) for key, value in inputs.items()}

            # Passar pelo modelo
            outputs = self.model(**inputs)

            # Pooler output: já é o resumo global baseado no token [CLS]
            pooler_output = outputs.pooler_output.cpu()  # Já é um resumo do texto

            # Fazer a média dos embeddings de todos os tokens (global pooling)
            output_mean_embedding = torch.mean(outputs.last_hidden_state, dim=1)
            output_mean_embedding = output_mean_embedding.cpu()

            # Sincronizar CUDA para garantir que a GPU terminou o processamento antes de prosseguir
            torch.cuda.synchronize()

            # Limpar variáveis não utilizadas e liberar memória da GPU
            del inputs, outputs
            torch.cuda.empty_cache()

            return {"bertimbau_pooler_output": pooler_output, "bertimbau_output_mean": output_mean_embedding}

bertimbau_embedding = BertimbauEmbedding()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

**Passo 3**

Vamos criar um *dataset* apenas com os embeddings para facilitar o processo, vamor iterar sobre cada instância do *dataframe* original e para cada uma executaremos o processo de embedding utilizando a classe declarada no passo anterior.

In [5]:
dataset = {'pooler_output' : [], 'output_mean' : []}

for i in range(len(df[2]['data'])):
    text = df[2]['data'][i]['call'] # Vamos fazer o embedding a partir da chamada da notícia "call"
    embedding = bertimbau_embedding.process_text(text)
    dataset['pooler_output'].append(embedding['bertimbau_pooler_output'])
    dataset['output_mean'].append(embedding['bertimbau_output_mean'])


**Passo 4**

Precisamos converter os tensores e reduzir a dimensionalidade da lista retornada no padrão do PyTorch. Primeiro transformamos *tensor* para *numpy*, depois realizamos o *squeeze*. Essa operação será realizada para as duas formas de embedding: *pooler output* e *output mean*.

In [10]:
import numpy as np

def transform_embedding(embedding):
  embedding = np.array(embedding) # Converte para numpy
  embedding = np.squeeze(embedding, axis=1) # Reduz a dimensionalidade da lista

  return embedding

for key in dataset.keys():
  dataset[key] = transform_embedding(dataset[key])

**Passo 5**

Criamos a função que aplicará o *cossine similarity* para retornar os índices dos textos mais semelhantes.

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

def cosine_recommend(embeddings, index, top_n=8):
    similarities = cosine_similarity([embeddings[index]], embeddings)  # Calcula a similaridade
    ranked_indices = np.argsort(similarities[0])[::-1]  # Ordena por similaridade em ordem decrescente

    # Exclui o próprio índice da lista de recomendações
    ranked_indices = [i for i in ranked_indices if i != index]

    # Retorna os top_n primeiros índices
    return ranked_indices[:top_n]

print("pooler output:")
print(cosine_recommend(dataset['pooler_output'], 0))
print("\noutput mean:")
print(cosine_recommend(dataset['output_mean'], 0))

pooler output:
[77, 128, 26, 50, 68, 166, 97, 62]

output mean:
[2, 224, 97, 1, 88, 63, 257, 93]


**Passo 6**

Para podermos visualizar o resultado vamos retornar uma lista dos resumos das matérias recomendadas antecedida pelo resumo da matéria escolhida.

In [22]:
def recommend(index, embedding='output_mean', top_n=8):
  indexes = cosine_recommend(dataset[embedding], index, top_n)

  rec = []

  for i in indexes:
    rec.append(df[2]['data'][i]['call'])

  return rec

def display_rec(index, embedding='output_mean', top_n=8):
  print(">>> SELECIONADO:")
  print(df[2]['data'][index]['call'],'\n')

  print(">>> RECOMENDAÇÕES:")
  for n in recommend(index, embedding, top_n):
    print(n,'\n')

display_rec(0)

>>> SELECIONADO:
Descubra todas as denúncias feitas pelo vereador Adriano Jiló a respeito do grave problema de captação e escassez de água no Ribeirão Canta Galo. Conheça os detalhes sobre os desafios enfrentados e as medidas necessárias para preservar nosso valioso recurso natural. 

>>> RECOMENDAÇÕES:
Adriano Jiló denuncia a escassez de água e o assoreamento na Cachoeira do Flávio. Ele cobra ações urgentes das autoridades para preservar as nascentes e resolver o problema. O vereador destaca a importância da conscientização e do cuidado com o meio ambiente. 

Desde 2018, vereadores de São Tomé das Letras denunciam problemas de captação e escassez de água no Ribeirão Canta Galo. Apesar de propostas para mitigar o assoreamento e melhorar a captação, a Prefeitura e a COPASA não responderam nem tomaram medidas. 

Grave incêndio ameaça mata ciliar em Sobradinho. Conheça os impactos devastadores desse desastre ambiental, saiba como as autoridades estão agindo e descubra como você pode ajuda

**Passo 7**

Por último que tal aplicar o mesmo processo em matérias aleatórias para provar que o processo não funciona apenas em uma situação específica, mas sim se aplica a todo o banco de dados.

In [28]:
import random

i = 1
for n in [random.randint(0, len(df[2]['data']) - 1) for _ in range(8)]:
  print(f"-------- TESTE {i} --------")
  i+=1
  display_rec(n)
  print("\n")

-------- TESTE 1 --------
>>> SELECIONADO:
São Tomé das Letras, conhecida por sua riqueza natural e mística, também é famosa na ufologia. Moradores e turistas relatam avistamentos de OVNIs e criaturas. A cidade, sobre um maciço de quartzito, atrai fenômenos por suas propriedades 'energéticas'. Um programa de ufologia, 'CASE arquivo UFO', traz esses casos ao público. 

>>> RECOMENDAÇÕES:
Via Fanzine e DTV lançam "CASE arquivo UFO", um programa mensal de ufologia. A estreia em 24/03 abordará um caso ufológico de 1982 em Campo Grande. Produzido por Pepe Chaves e Dante Villarruel, o programa promete trazer casos desconhecidos, com acesso livre ao público. 

São Tomé das Letras é uma cidade mística, cercada por lendas e uma rica história que data do século XVIII. Conhecida por sua localização na Serra da Mantiqueira e arquitetura única, atrai por suas belezas naturais e mistérios, como a estátua de São Tomé encontrada numa gruta e inscrições antigas indecifráveis. Fenômenos estranhos e avis